In [23]:
!pip install fastapi uvicorn edge-tts faster-whisper requests transformers torch

1131.81s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached torch-2.7.1-cp310-none-macosx_11_0_arm64.whl.metadata (29 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (4.0 kB)
Using cached torch-2.7.1-cp310-none-macosx_11_0_arm64.whl (68.6 MB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached MarkupSafe-3.0.2-cp310-cp310-macosx_11_0_arm64.whl (12 kB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [torch]32m3/4 [torch]


In [30]:
import torch
from transformers import pipeline

# 初始化 LLaMA3 pipeline（你必须有访问权 + 模型文件）
llm = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.1-8B",
    max_new_tokens=256,
    do_sample=True
)


@app.post("/chat/")
async def chat(file: UploadFile = File(...)):
    try:
        # 保存上传音频
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
            tmp.write(await file.read())
            input_path = tmp.name

        # Step 1: 语音转文字
        segments, _ = model.transcribe(input_path)
        user_text = "".join([s.text for s in segments]).strip()

        if not user_text:
            raise Exception("语音识别结果为空，可能是音频不清晰或格式问题")

        # Step 2: 构造上下文提示词
        conversation_history.append(f"User: {user_text}")
        prompt = "\n".join(conversation_history) + "\nAssistant:"

        # 调用本地 LLM 生成回答
        output = llm(prompt)[0]['generated_text']
        reply = output[len(prompt):].strip().split("\n")[0]  # 取首句回答
        conversation_history.append(f"Assistant: {reply}")

        # Step 3: TTS 合成回复音频
        output_path = input_path.replace(".wav", "_response.wav")
        await synthesize(reply, output_path)

        return FileResponse(output_path, media_type="audio/wav", filename="response.wav")

    except Exception as e:
        print("❌ 服务出错:", e)
        raise HTTPException(status_code=500, detail=str(e))

NameError: name 'torch' is not defined